In [1]:
with open('input', 'r') as ifile:
    _rules, _myticket, _tickets = ifile.read().split('\n\n')
rules = {field: [(int(low), int(upp)) for low, upp in (_range.split('-') for _range in r.split(' or '))]
         for field, r in (row.split(': ') for row in _rules.split('\n'))}
myticket = [int(val) for val in _myticket.split('\n')[1].split(',')]
tickets = [[int(val) for val in ticket.split(',')] for ticket in _tickets.splitlines()[1:]]

# Part 1

In [2]:
sum(val for ticket in tickets for val in ticket
    if not any(_min <= val <= _max for rule in rules for _min, _max in rules[rule]))

29878

# Part 2

In [3]:
valid_tickets = [ticket for ticket in tickets
                 if all(any(_min <= val <= _max for rule in rules for _min, _max in rules[rule])
                        for val in ticket)]

In [4]:
def explore(options, i):
    already_visited = sum((previous_choice for _, previous_choice in options[:i]), [])
    return [(options[:i]
             + [(options[i][0], [explored_choice])]
             + [(k, [choice for choice in choices if choice not in already_visited + [explored_choice]])
                for k, choices in options[i + 1:]])
            for explored_choice in options[i][1]]

In [5]:
def derive_mapping(options):
    if not all(choices for _, choices in options):
        return None    
    if all(len(choices) == 1 for _, choices in options):
        if len(set(_mapping[0] for _, _mapping in options for field in _mapping)) < len(options):
            return None
        return [(j, choices[0]) for j, choices in options]
    i = next(i for i, (_, choices) in enumerate(options) if len(choices) > 1)
    return next((mapping for mapping in (derive_mapping(_options)
                                         for _options in explore(options, i))
                 if mapping),
                None)

In [6]:
options = [(j, [field_name for field_name, ranges in rules.items()
            if all(any(_min <= ticket[j] <= _max for _min, _max in ranges) for ticket in valid_tickets)])
           for j in range(len(tickets[0]))]

In [7]:
from functools import reduce

In [8]:
reduce(lambda x, y: x * y,
       (myticket[i] for i, field in derive_mapping(sorted(options, key=lambda t: len(t[1])))
        if field.startswith('departure')))

855438643439